<a href="https://colab.research.google.com/github/keysenelvial/dinam/blob/main/dinam3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №3**

Выберите временной ряд цен акций.
Подготовьте данные для обучения моделей машинного обучения (нормализация, создание признаков).
Разделите данные на обучающую и тестовую выборки. Реализуйте и обучите следующие модели: a) Логистическая регрессия b) Машина опорных векторов c) Модель случайного леса (Random Forest) d) Градиентный бустинг (например, XGBoost) Настройте гиперпараметры моделей с помощью кросс-валидации. Сделайте прогноз на тестовой выборке для каждой модели. Сравните результаты прогнозирования моделей между собой и с простыми методами (например, наивный прогноз). Визуализируйте результаты прогнозирования. Проанализируйте важность признаков для моделей, где это применимо. Сделайте выводы о эффективности различных подходов машинного обучения в прогнозировании цен акций.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from xgboost import XGBRegressor

# Загрузка и предобработка данных
data = pd.read_csv("/content/drive/MyDrive/AMZN.csv", sep=";", header=None, names=["DATE", "TIME", "OPEN", "HIGH", "LOW", "CLOSE", "VOL"])
data['DATETIME'] = pd.to_datetime(data['DATE'].astype(str) + data['TIME'].astype(str), format='%y%m%d%H%M%S')
data.sort_values(by="DATETIME", inplace=True)
data.set_index("DATETIME", inplace=True)

# Выбираем только цены закрытия для анализа
ts = data['CLOSE']
print(ts.head())

DATETIME
2024-07-31 09:30:00    185.27
2024-07-31 09:31:00    185.37
2024-07-31 09:32:00    184.99
2024-07-31 09:33:00    184.88
2024-07-31 09:34:00    184.60
Name: CLOSE, dtype: float64


In [ ]:
# Нормализация данных
scaler = MinMaxScaler()
ts_scaled = scaler.fit_transform(ts.values.reshape(-1, 1))

# 2. Создание признаков
def create_features(data, look_back=60):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

look_back = 60
X, y = create_features(ts_scaled, look_back=look_back)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


Лучшие параметры (логистическая регрессия): {'logistic__C': 0.23357214690901212, 'logistic__solver': 'liblinear'}
Лучшая оценка: 0.47819110138584986


In [ ]:
# 3. Реализация моделей

# 3.1 Логистическая регрессия
# Преобразуем задачу в классификацию направления движения цены
y_direction = (y_train > np.roll(y_train, 1))[1:].astype(int)
X_train_lr = X_train[1:]

lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train_lr.reshape(X_train_lr.shape[0], -1), y_direction)

# 3.2 Машина опорных векторов (SVM)
svm_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svm_model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

# 3.3 Случайный лес (Random Forest)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

# 3.4 XGBoost
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

# 4. Оценка моделей
def evaluate_model(model, X_test, y_test, model_name):
    if model_name == 'Logistic Regression':
        # Для логистической регрессии прогнозируем направление движения
        y_pred_direction = model.predict(X_test.reshape(X_test.shape[0], -1))
        y_pred = np.where(y_pred_direction == 1, y_test + 0.01, y_test - 0.01)
    else:
        y_pred = model.predict(X_test.reshape(X_test.shape[0], -1)).reshape(-1, 1)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)

    print(f'{model_name} - MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}')
    return y_pred

lr_pred = evaluate_model(lr_model, X_test, y_test, 'Logistic Regression')
svm_pred = evaluate_model(svm_model, X_test, y_test, 'SVM')
rf_pred = evaluate_model(rf_model, X_test, y_test, 'Random Forest')
xgb_pred = evaluate_model(xgb_model, X_test, y_test, 'XGBoost')

# 5. Визуализация результатов
plt.figure(figsize=(12, 6))
plt.plot(ts.index[-len(y_test):], scaler.inverse_transform(y_test), label='Actual', color='black')
plt.plot(ts.index[-len(y_test):], scaler.inverse_transform(lr_pred), label='Logistic Regression', alpha=0.7)
plt.plot(ts.index[-len(y_test):], scaler.inverse_transform(svm_pred), label='SVM', alpha=0.7)
plt.plot(ts.index[-len(y_test):], scaler.inverse_transform(rf_pred), label='Random Forest', alpha=0.7)
plt.plot(ts.index[-len(y_test):], scaler.inverse_transform(xgb_pred), label='XGBoost', alpha=0.7)
plt.title('Сравнение прогнозов моделей машинного обучения')
plt.xlabel('Дата')
plt.ylabel('Цена акции')
plt.legend()
plt.show()

# 6. Анализ важности признаков (для Random Forest)
feature_importance = pd.DataFrame({'feature': range(X_train.shape[1]), 'importance': rf_model.feature_importances_})
feature_importance = feature_importance.sort_values('importance', ascending=False).head(10)

plt.figure(figsize=(10, 6))
plt.bar(feature_importance['feature'], feature_importance['importance'])
plt.title('Топ-10 важных признаков (Random Forest)')
plt.xlabel('Индекс признака')
plt.ylabel('Важность')
plt.show()